# Read in results

In [1]:
from sklearn.metrics import roc_auc_score
import sys
sys.path.append('text_nn')
import pandas as pd
import rationale_net
from rationale_net.datasets.new_york_times_dataset import NYTimesDataset
import pickle
results = pickle.load(open("exploratory_analysis/logs/results__model-form_cnn__num-layers_1__word-cutoff_120__epochs_30", 'rb'))

In [2]:
pd.Series(results)

train                                                              True
test                                                               True
cuda                                                               True
num_gpus                                                              1
debug_mode                                                        False
class_balance                                                     False
objective                                                 cross_entropy
aspect                                                          overall
init_lr                                                          0.0001
epochs                                                               30
batch_size                                                           64
patience                                                              5
tuning_metric                                                      loss
save_dir                                                       s

In [3]:
test_stats = pd.Series(results['test_stats'])

True ratio

In [4]:
pd.Series(results['test_stats']['golds']).value_counts()

0    246089
1     11379
dtype: int64

Predicted ratio

In [5]:
pd.Series(results['test_stats']['preds']).value_counts()

0    159708
1     97760
dtype: int64

In [6]:
y_proba = list(map(lambda x: x[1] if x[0] == 1 else 1-x[1], zip(test_stats['preds'], test_stats['probas'])))
y_true = results['test_stats']['golds']

ROC

In [8]:
roc_auc_score(y_true, y_proba)

0.6525972087636016

# Rationals

In [9]:
rationales = results['test_stats']['rationales']

In [10]:
rationales[4]

'_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ cazale _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _'

In [11]:
rationale_words = pd.Series(list(map(lambda r: ' '.join(r.replace('_', '').split()), rationales)))

lengths of rational

In [19]:
rationale_words.str.split(' ').str.len().value_counts().to_frame('count')

,count
1,249532
2,7103
3,706
4,103
5,18
6,5
7,1


In [13]:
rational_df = pd.concat([
    rationale_words.to_frame('word_list'),
    pd.Series(y_true).to_frame('y_true'),
    pd.Series(results['test_stats']['preds']).to_frame('y_pred')
], axis=1)

Overall most popular rationales

In [15]:
rational_df['word_list'].value_counts().head(10)

nyt          8013
-            5480
:            2295
yesterday    1513
editor       1505
tuesday      1408
             1354
bush         1326
monday       1321
wednesday    1229
Name: word_list, dtype: int64

Popular two-word rationales

In [16]:
rational_df['word_list'].loc[lambda s: s.str.split().str.len()==2].value_counts().head(10)

nyt nyt                295
mourns mourns          225
kristof kristof        134
krugman krugman         91
mourn offit             83
swarns nyt              59
safire safire           58
mcgreevey mcgreevey     53
dugger nyt              40
arenson nyt             27
Name: word_list, dtype: int64

Popular rationals for y_pred = 1 (article = front page)

In [18]:
rational_df.loc[lambda df: df['y_pred'] == 1]['word_list'].value_counts().head(10)

nyt          7244
bush         1326
pataki        637
nation        426
old           413
sgt           400
n.b.a.        375
city          334
officials     318
nyt nyt       295
Name: word_list, dtype: int64

Popular rationales for y_pred = 0 (article != front page)

In [17]:
rational_df.loc[lambda df: df['y_pred'] == 0]['word_list'].value_counts().head(10)

-            5480
:            2295
yesterday    1513
editor       1505
tuesday      1408
             1354
monday       1321
wednesday    1229
thursday     1176
sunday       1094
Name: word_list, dtype: int64